En este notebook, para cada estancia en UCI guardada en MIMIC-III, extraemos los datos de diferentes variables que componen OASIS (Oxford acute severity of illness score) relacionados con esa estancia y a partir de estos datos calculamos la puntuación OASIS de esa estancia.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.stats as stats
import pylab
import seaborn as sns
from tqdm import tqdm
import csv
matplotlib.style.use('ggplot')

In [3]:
patients=pd.read_csv('/data/demo/PATIENTS.csv')
print(patients.shape)
print('unique SUBJECT_ID:', patients.SUBJECT_ID.nunique())
patients.head()

(100, 8)
unique SUBJECT_ID: 100


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1


In [19]:
admissions=pd.read_csv('/data/demo/ADMISSIONS.csv')
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
admissions.head()

(129, 19)
unique SUBJECT_ID: 100
unique HADM_ID   : 129


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,SEPSIS,1,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,NaN,NaN,ALCOHOLIC HEPATITIS,1,1


In [20]:
admissions=admissions.drop(['ROW_ID', 'DEATHTIME', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'LANGUAGE', 
                        'RELIGION', 'MARITAL_STATUS', 'EDREGTIME', 'EDOUTTIME'], axis=1)

In [21]:
admissions=pd.merge(patients, admissions, on=['SUBJECT_ID'])
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
admissions.head()

(129, 17)
unique SUBJECT_ID: 100
unique HADM_ID   : 129


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,INSURANCE,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,Medicare,BLACK/AFRICAN AMERICAN,SEPSIS,0,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,EMERGENCY,Private,UNKNOWN/NOT SPECIFIED,HEPATITIS B,1,1
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,EMERGENCY,Medicare,UNKNOWN/NOT SPECIFIED,SEPSIS,1,1
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,EMERGENCY,Medicare,WHITE,HUMERAL FRACTURE,0,1
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,EMERGENCY,Medicare,WHITE,ALCOHOLIC HEPATITIS,1,1


In [22]:
icustays=pd.read_csv('/data/demo/ICUSTAYS.csv')
print(icustays.shape)
print('unique SUBJECT_ID:', icustays.SUBJECT_ID.nunique())
print('unique HADM_ID   :', icustays.HADM_ID.nunique())
print('unique ICUSTAY_ID:', icustays.ICUSTAY_ID.nunique())
icustays.head()

(136, 12)
unique SUBJECT_ID: 100
unique HADM_ID   : 129
unique ICUSTAY_ID: 136


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,12742,10006,142345,206504,carevue,MICU,MICU,52,52,2164-10-23 21:10:15,2164-10-25 12:21:07,1.6325
1,12747,10011,105331,232110,carevue,MICU,MICU,15,15,2126-08-14 22:34:00,2126-08-28 18:59:00,13.8507
2,12749,10013,165520,264446,carevue,MICU,MICU,15,15,2125-10-04 23:38:00,2125-10-07 15:13:52,2.6499
3,12754,10017,199207,204881,carevue,CCU,CCU,7,7,2149-05-29 18:52:29,2149-05-31 22:19:17,2.1436
4,12755,10019,177759,228977,carevue,MICU,MICU,15,15,2163-05-14 20:43:56,2163-05-16 03:47:04,1.2938


In [23]:
icustays=icustays.drop(['ROW_ID', 'LAST_CAREUNIT', 'LAST_WARDID'], axis=1)

In [24]:
admissions=pd.merge(admissions, icustays, on=['SUBJECT_ID', 'HADM_ID'])
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions.ICUSTAY_ID.nunique())
admissions.head()

(136, 24)
unique SUBJECT_ID: 100
unique HADM_ID   : 129
unique ICUSTAY_ID: 136


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,...,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,FIRST_WARDID,INTIME,OUTTIME,LOS
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1,142345,2164-10-23 21:09:00,...,SEPSIS,0,1,206504,carevue,MICU,52,2164-10-23 21:10:15,2164-10-25 12:21:07,1.6325
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1,105331,2126-08-14 22:32:00,...,HEPATITIS B,1,1,232110,carevue,MICU,15,2126-08-14 22:34:00,2126-08-28 18:59:00,13.8507
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1,165520,2125-10-04 23:36:00,...,SEPSIS,1,1,264446,carevue,MICU,15,2125-10-04 23:38:00,2125-10-07 15:13:52,2.6499
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1,199207,2149-05-26 17:19:00,...,HUMERAL FRACTURE,0,1,204881,carevue,CCU,7,2149-05-29 18:52:29,2149-05-31 22:19:17,2.1436
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1,177759,2163-05-14 20:43:00,...,ALCOHOLIC HEPATITIS,1,1,228977,carevue,MICU,15,2163-05-14 20:43:56,2163-05-16 03:47:04,1.2938


In [25]:
#we compute the age at the time the patient is admitted to the ICU from date of birth dob, then we drop dob.
admissions['DOB'] = pd.to_datetime(admissions['DOB']).dt.date
admissions['INTIME_TMP'] = pd.to_datetime(admissions['INTIME']).dt.date
admissions['AGE'] = admissions.apply(lambda e: (e['INTIME_TMP'] - e['DOB']).days/365, axis=1)
admissions=admissions.drop(['DOB'], axis=1)
admissions=admissions.drop(['INTIME_TMP'], axis=1)

In [26]:
admissions=admissions.drop(['INSURANCE','ETHNICITY'], axis=1)
admissions=admissions.drop(['DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG','DISCHTIME','DIAGNOSIS','HOSPITAL_EXPIRE_FLAG','HAS_CHARTEVENTS_DATA','FIRST_CAREUNIT','FIRST_WARDID','OUTTIME'], axis=1)

In [27]:
#we compute the  Pre-ICU length of stay in hours. 
prelos_tmp= pd.to_datetime(admissions['INTIME'])-pd.to_datetime(admissions['ADMITTIME'])
admissions['PRELOS']=np.round(prelos_tmp.dt.total_seconds()/3600, 1)
admissions['PRELOS']=np.where(admissions['PRELOS']<0,0,admissions['PRELOS']) #hours, (la fila 128 sale negativo, error de introducir admittime y intime, )


In [28]:
older89=len(admissions[(admissions['AGE']>=300)])
older89

9

In [29]:
admissions['AGE']=admissions['AGE'].apply(lambda x: np.random.triangular(89.01, 89.01, 100) if x >= 300 else x)
admissions.shape


(136, 12)

In [14]:
#prueba_merged=pd.merge(admissions.iloc[0].to_frame().T,df_temp,how='left',on=['subject_id', 'hadm_id','icustay_id'])
#prueba_merged.head()

In [30]:
#para hacer prueba de prints, coger un solo icustay_id:
#admissions = admissions.loc[admissions['ICUSTAY_ID'] == 264446]  #icustay_id que tiene datos en los 10 variables.
#admissions

In [31]:
admissions=admissions.drop(['ROW_ID','GENDER','DBSOURCE'], axis=1)

In [32]:
admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,ADMISSION_TYPE,ICUSTAY_ID,INTIME,LOS,AGE,PRELOS
0,10006,142345,2164-10-23 21:09:00,EMERGENCY,206504,2164-10-23 21:10:15,1.6325,70.682192,0.0
1,10011,105331,2126-08-14 22:32:00,EMERGENCY,232110,2126-08-14 22:34:00,13.8507,36.213699,0.0
2,10013,165520,2125-10-04 23:36:00,EMERGENCY,264446,2125-10-04 23:38:00,2.6499,87.142466,0.0
3,10017,199207,2149-05-26 17:19:00,EMERGENCY,204881,2149-05-29 18:52:29,2.1436,73.734247,73.6
4,10019,177759,2163-05-14 20:43:00,EMERGENCY,228977,2163-05-14 20:43:56,1.2938,48.931507,0.0
...,...,...,...,...,...,...,...,...,...
131,44083,198330,2112-05-28 15:45:00,EMERGENCY,286428,2112-05-29 02:01:33,3.6174,54.569863,10.3
132,44154,174245,2178-05-14 20:29:00,EMERGENCY,217724,2178-05-14 20:29:55,0.6259,98.035910,0.0
133,44212,163189,2123-11-24 14:14:00,EMERGENCY,239396,2123-11-24 14:14:29,31.1235,45.468493,0.0
134,44222,192189,2180-07-19 06:55:00,EMERGENCY,238186,2180-07-19 06:56:38,1.3279,73.112329,0.0


In [33]:
import os

In [37]:
output_path = 'eficienteV3_3_demo/data/root/'
try:
    os.makedirs(output_path)
except:
    pass

In [38]:
admissions.to_csv(os.path.join(output_path, 'all_stays.csv'), index=False)

In [39]:
def break_up_stays(stays, output_path):
    nb_stays = stays.shape[0]
    stays_ids = stays.ICUSTAY_ID.unique()
    for icustay_id in tqdm(stays_ids, total=nb_stays, desc='Breaking up stays'):
        dn = os.path.join(output_path, str(icustay_id))
        try:
            os.makedirs(dn)
        except:
            pass

        stays[stays.ICUSTAY_ID == icustay_id].to_csv(os.path.join(dn, 'stays.csv'),
                                                                              index=False)


In [40]:
def read_events_table_by_row(mimic3_path, table):
    nb_rows = {'chartevents': 330712484,  'outputevents': 4349219, 'procedureevents_mv': 258067,}
    reader = csv.DictReader(open(os.path.join(mimic3_path, table.upper() + '.csv'), 'r'))
    for i, row in enumerate(reader):
        if 'ICUSTAY_ID' not in row:
            row['ICUSTAY_ID'] = ''
        yield row, i, nb_rows[table.lower()]


In [41]:
def read_events_table_and_break_up_by_stays(mimic3_path, table, output_path,
                                              items_to_keep=None, stays_to_keep=None):
    
    if(table == 'PROCEDUREEVENTS_MV'):
        obs_header = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ITEMID']
    
    elif(table == 'OUTPUTEVENTS'):
        obs_header = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE','VALUEUOM','ISERROR']
    
    else:
        obs_header = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM','VALUEUOM','ERROR']

    
    if items_to_keep is not None:
        items_to_keep = set([str(s) for s in items_to_keep])

    if stays_to_keep is not None:
        stays_to_keep = set([str(s) for s in stays_to_keep])
        
    class DataStats(object):
        def __init__(self):
            self.curr_icustay_id = ''
            self.curr_obs = []

    data_stats = DataStats()

    def write_current_observations():
        dn = os.path.join(output_path, str(data_stats.curr_icustay_id))
        try:
            os.makedirs(dn)
        except:
            pass
        
        if(table == 'PROCEDUREEVENTS_MV'):
            fn = os.path.join(dn, 'ventilation.csv')
        elif(table == 'OUTPUTEVENTS'):
            fn = os.path.join(dn, 'urine.csv')
        else:
            fn = os.path.join(dn, 'events.csv')
        
        
        if not os.path.exists(fn) or not os.path.isfile(fn):
            f = open(fn, 'w')
            f.write(','.join(obs_header) + '\n')
            f.close()
        w = csv.DictWriter(open(fn, 'a'), fieldnames=obs_header, quoting=csv.QUOTE_MINIMAL)
        w.writerows(data_stats.curr_obs)
        data_stats.curr_obs = []

    nb_rows_dict = {'chartevents': 330712484, 'outputevents': 4349219, 'procedureevents_mv': 258067}
    nb_rows = nb_rows_dict[table.lower()]

    for row, row_no, _ in tqdm(read_events_table_by_row(mimic3_path, table), total=nb_rows,
                                                        desc='Processing {} table'.format(table)):

        if (stays_to_keep is not None) and (row['ICUSTAY_ID'] not in stays_to_keep):
            continue
        if (items_to_keep is not None) and (row['ITEMID'] not in items_to_keep):
            continue        
        
        if(table == 'PROCEDUREEVENTS_MV'):
            row_out = {'SUBJECT_ID': row['SUBJECT_ID'],
                       'HADM_ID': row['HADM_ID'],
                       'ICUSTAY_ID': '' if 'ICUSTAY_ID' not in row else row['ICUSTAY_ID'],
                       'STARTTIME': row['STARTTIME'],
                       'ITEMID': row['ITEMID']}
        
        elif(table == 'OUTPUTEVENTS'):
            row_out = {'SUBJECT_ID': row['SUBJECT_ID'],
                       'HADM_ID': row['HADM_ID'],
                       'ICUSTAY_ID': '' if 'ICUSTAY_ID' not in row else row['ICUSTAY_ID'],
                       'CHARTTIME': row['CHARTTIME'],
                       'ITEMID': row['ITEMID'],
                       'VALUE': row['VALUE'],
                       'VALUEUOM': row['VALUEUOM'],
                        'ISERROR': row['ISERROR']}
        
        else:      
            row_out = {'SUBJECT_ID': row['SUBJECT_ID'],
                       'HADM_ID': row['HADM_ID'],
                       'ICUSTAY_ID': '' if 'ICUSTAY_ID' not in row else row['ICUSTAY_ID'],
                       'CHARTTIME': row['CHARTTIME'],
                       'ITEMID': row['ITEMID'],
                       'VALUE': row['VALUE'],
                       'VALUENUM': row['VALUENUM'],
                       'VALUEUOM': row['VALUEUOM'],
                       'ERROR': row['ERROR']}
            
        if data_stats.curr_icustay_id != '' and data_stats.curr_icustay_id != row['ICUSTAY_ID']:
            write_current_observations()
        data_stats.curr_obs.append(row_out)
        data_stats.curr_icustay_id = row['ICUSTAY_ID']

    if data_stats.curr_icustay_id != '':
        write_current_observations()

In [42]:
break_up_stays(admissions, output_path)

Breaking up stays: 100%|██████████████████████| 136/136 [00:00<00:00, 262.71it/s]


In [43]:
itemids = [223762,223761,676,677,678,679,220045,211,220210,615,618,220052,220181,225312,456,52,6702,443,220739,184,223901,454,223900,723,198,225792,225794,40055,226559,40069,226560,40094,40715,40473,40085,40057,40056,40405,40428,40086,40096,40651,226561,226584,226563,226564,226565,226567,226557,226558]

In [ ]:
"""
https://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/gcs_first_day.sql

723  Verbal Response
454  Motor Response
184  Eye Opening
198  GCS Total

223900 GCS - Verbal Response
223901 GCS - Motor Response
220739 GCS - Eye Opening

https://github.com/MIT-LCP/mimic-code/blob/master/concepts/cookbook/temp.sql
676 -- Temperature C
677 -- Temperature C (calc)
678 -- Temperature F
679 -- Temperature F (calc)
223761 -- Temperature Fahrenheit
223762 -- Temperature Celsius


#TODO: review ventilation itemIDs
https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation_classification.sql
¿¿226260 Mechanically Ventilated?? 
Si en chartevents.csv hay 720, 223849 -- Ventilator Mode, 223848 -- Ventilator Type
indica que ese ICUSTAY_ID fue ventilated?
Actual:
225792	Invasive Ventilation
225794	Non-invasive Ventilation




https://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/vitals_first_day.sql

220045,211  HeartRate

MAP:
456	NBP Mean
52 Arterial BP Mean
6702 Arterial BP Mean #2
443 Manual BP Mean(calc)
220052 Arterial Blood Pressure mean
220181 Non Invasive Blood Pressure mean
225312 ART BP mean

RespRate:
615 Resp Rate (Total)
618 Respiratory Rate
220210 Respiratory Rate
224690 Respiratory Rate (Total)



https://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/urine_output_first_day.sql
#TODO: revisar? ponerlos todos?
urine(todo en mL):
-- these are the most frequently occurring urine output observations in CareVue
40055, -- "Urine Out Foley"
43175, -- "Urine ."  <--- no uso
40069, -- "Urine Out Void"
40094, -- "Urine Out Condom Cath"
40715, -- "Urine Out Suprapubic"
40473, -- "Urine Out IleoConduit"
40085, -- "Urine Out Incontinent"
40057, -- "Urine Out Rt Nephrostomy"
40056, -- "Urine Out Lt Nephrostomy"
40405, -- "Urine Out Other"
40428, -- "Urine Out Straight Cath"
40086,--   Urine Out Incontinent
40096, -- "Urine Out Ureteral Stent #1"
40651, -- "Urine Out Ureteral Stent #2"

-- these are the most frequently occurring urine output observations in MetaVision
226559, -- "Foley"
226560, -- "Void"
226561, -- "Condom Cath"
226584, -- "Ileoconduit"
226563, -- "Suprapubic"
226564, -- "R Nephrostomy"
226565, -- "L Nephrostomy"
226567, --  Straight Cath
226557, -- R Ureteral Stent
226558, -- L Ureteral Stent

"""

In [44]:
event_tables = ['PROCEDUREEVENTS_MV','OUTPUTEVENTS','CHARTEVENTS']

In [46]:
mimic3_path = '/data/demo/'

In [47]:
#Esta celda tarda mucho, 1h aprox (para 10000 icustay_id)
#Ejecutar solo una vez
items_to_keep = set([int(itemid) for itemid in itemids]) 
for table in event_tables:
    read_events_table_and_break_up_by_stays(mimic3_path, table, output_path, items_to_keep=items_to_keep,stays_to_keep=admissions.ICUSTAY_ID.unique())

Processing PROCEDUREEVENTS_MV table:   0%| | 753/258067 [00:00<02:31, 1695.91it/s
Processing OUTPUTEVENTS table:   0%|  | 11320/4349219 [00:00<01:46, 40691.49it/s]
Processing CHARTEVENTS table:   0%| | 758355/330712484 [00:06<48:51, 112551.33it/


### Ahora vamos a sacar todos los datos de los variables requeridos del events,urine, ventilation, para luego mergearlos con la  información general del paciente: 

In [48]:
stays_root_path = 'eficienteV3_3_demo/data/root/'

In [49]:
def read_stays(subject_path):
    stays = pd.read_csv(os.path.join(subject_path, 'stays.csv'))
    return stays

def read_ventilation(subject_path):
    fn = os.path.join(subject_path, 'ventilation.csv')
    if not os.path.exists(fn) or not os.path.isfile(fn):
        ventilations = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','STARTTIME'])         
    else:
        ventilations = pd.read_csv(fn)
    return ventilations

def read_urine(subject_path):
    fn = os.path.join(subject_path, 'urine.csv')
    if not os.path.exists(fn) or not os.path.isfile(fn):
        urine = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','URINE_TIME','URINE']) #create empty dataframe with columns name URINE    
    else:
        urine = pd.read_csv(fn)
        urine=urine[urine['ISERROR'].isnull()]
        urine=urine.rename({'CHARTTIME':'URINE_TIME', 'VALUE':'URINE'}, axis='columns')
        urine = urine.drop(['ITEMID','VALUEUOM'],axis=1)

    return urine

In [50]:
def read_events(subject_path):
    fn = os.path.join(subject_path, 'events.csv')
    if not os.path.exists(fn) or not os.path.isfile(fn):
        df_temp = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','TEMP_C_TIME','TEMP_C'])
        df_hrate = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','HRATE_TIME','HRATE'])
        df_resp_rate = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','RESP_RATE_TIME','RESP_RATE'])
        df_MAP = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','MAP_TIME','MAP'])
        
        df_GCS_E = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','GCS_E_TIME','GCS_E'])
        df_GCS_M = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','GCS_M_TIME','GCS_M'])
        df_GCS_V = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','GCS_V_TIME','GCS_V'])
        df_GCS_TOTAL_CAREVUE = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','GCS_TOTAL_CAREVUE_TIME','GCS_TOTAL_CAREVUE'])
    else:
        events = pd.read_csv(fn)
        events.fillna(value=np.nan, inplace=True)
        
        #filtrar filas con error
        events = events.loc[(events['ERROR'] == 0.0) | (events['ERROR'].isna()) ]
        
        df_temp_C = events.loc[events['ITEMID'].isin([223762,676,677])]
        df_temp_C=df_temp_C.rename({'CHARTTIME':'TEMP_C_TIME', 'VALUE':'TEMP_C'}, axis='columns')

        
        df_temp_F = events.loc[events['ITEMID'].isin([223761,678,679])].copy()
        df_temp_F['VALUE'] = pd.to_numeric(df_temp_F['VALUE'])
        df_temp_F['VALUE'] = (df_temp_F['VALUE']- 32) * 5 / 9  #convert fahrenheit to celsius
        df_temp_F=df_temp_F.rename({'CHARTTIME':'TEMP_C_TIME', 'VALUE':'TEMP_C'}, axis='columns')
        
        df_temp = pd.concat([df_temp_C,df_temp_F])
        df_temp = df_temp.drop(['ITEMID','VALUEUOM','VALUENUM','ERROR'],axis=1)

        
        df_hrate = events.loc[events['ITEMID'].isin([220045,211])]
        df_hrate=df_hrate.rename({'CHARTTIME':'HRATE_TIME', 'VALUE':'HRATE'}, axis='columns')
        df_hrate = df_hrate.drop(['ITEMID','VALUEUOM','VALUENUM','ERROR'],axis=1)
        
        df_resp_rate = events.loc[events['ITEMID'].isin([220210,224690,615,618])]
        df_resp_rate=df_resp_rate.rename({'CHARTTIME':'RESP_RATE_TIME', 'VALUE':'RESP_RATE'}, axis='columns')
        df_resp_rate = df_resp_rate.drop(['ITEMID','VALUEUOM','VALUENUM','ERROR'],axis=1)
        
        df_MAP = events.loc[events['ITEMID'].isin([220052,220181,225312,456,52,6702,443])]
        df_MAP=df_MAP.rename({'CHARTTIME':'MAP_TIME', 'VALUE':'MAP'}, axis='columns')
        df_MAP = df_MAP.drop(['ITEMID','VALUEUOM','VALUENUM','ERROR'],axis=1)

        df_GCS_E = events.loc[events['ITEMID'].isin([220739,184])]
        df_GCS_E = df_GCS_E.rename({'CHARTTIME':'GCS_E_TIME', 'VALUENUM':'GCS_E'}, axis='columns')
        df_GCS_E = df_GCS_E.drop(['ITEMID','VALUEUOM','VALUE','ERROR'],axis=1)

        
        df_GCS_M = events.loc[events['ITEMID'].isin([223901,454])]
        df_GCS_M=df_GCS_M.rename({'CHARTTIME':'GCS_M_TIME', 'VALUENUM':'GCS_M'}, axis='columns')
        df_GCS_M = df_GCS_M.drop(['ITEMID','VALUEUOM','VALUE','ERROR'],axis=1)

        
        df_GCS_V = events.loc[events['ITEMID'].isin([223900,723])]
        df_GCS_V=df_GCS_V.rename({'CHARTTIME':'GCS_V_TIME', 'VALUENUM':'GCS_V'}, axis='columns')
        df_GCS_V = df_GCS_V.drop(['ITEMID','VALUEUOM','VALUE','ERROR'],axis=1)

        
        df_GCS_TOTAL_CAREVUE = events.loc[events['ITEMID'] == 198]
        df_GCS_TOTAL_CAREVUE=df_GCS_TOTAL_CAREVUE.rename({'CHARTTIME':'GCS_TOTAL_CAREVUE_TIME', 'VALUENUM':'GCS_TOTAL_CAREVUE'}, axis='columns')
        df_GCS_TOTAL_CAREVUE = df_GCS_TOTAL_CAREVUE.drop(['ITEMID','VALUEUOM','VALUE','ERROR'],axis=1)
        
        
    return df_temp,df_hrate,df_resp_rate,df_MAP,df_GCS_E,df_GCS_M,df_GCS_V,df_GCS_TOTAL_CAREVUE


### For each variable, the worst score across the first day in ICU should be used to tabulate OASIS

In [51]:

# The MIT License

# Copyright (c) 2015 Tom Pollard

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

def compute_oasis(pd_dataframe):
    """
    Takes Pandas DataFrame as an argument and computes Oxford Acute 
    Severity of Illness Score (OASIS) (http://oasisicu.com/)
    
    The DataFrame should include only measurements taken over the first 24h 
    from admission. pd_dataframe should contain the following columns:
    
    'prelos' => Pre-ICU length of stay, hours
    'age' => Age of patient, years
    'GCS_total' => Total Glasgow Coma Scale for patient
    'hrate' => All heart rate measurements
    'MAP' => All mean arterial blood pressure measurements
    'resp_rate' => All respiratory rate measurements
    'temp_c' => All temperature measurements, C
    'urine' => Total urine output over 24 h (note, not consecutive measurements)
    'ventilated' => Is patient ventilated? (y,n)
    'admission_type' => Type of admission (elective, urgent, emergency) 
    
    Reference:
    Johnson AE, Kramer AA, Clifford GD. A new severity of illness scale 
    using a subset of Acute Physiology And Chronic Health Evaluation 
    data elements shows comparable predictive accuracy.
    Crit Care Med. 2013 Jul;41(7):1711-8. doi: 10.1097/CCM.0b013e31828a24fe
    http://www.ncbi.nlm.nih.gov/pubmed/23660729
    """
    
    # 10 variables
    oasis_score, oasis_prelos, oasis_age, oasis_gcs, oasis_hr, oasis_map, oasis_resp, \
        oasis_temp, oasis_urine, oasis_vent, oasis_surg = 0,0,0,0,0,0,0,0,0,0,0   
    # Pre-ICU length of stay, hours
    for val in pd_dataframe['PRELOS']:
        if val >= 4.95 and val <= 24.0:
            oasis_prelos = np.nanmax([0,oasis_prelos])
        elif val > 311.8:
            oasis_prelos = np.nanmax([1,oasis_prelos])
        elif val > 24.0 and val <= 311.8:
            oasis_prelos = np.nanmax([2,oasis_prelos])
        elif val >= 0.17 and val < 4.95:
            oasis_prelos = np.nanmax([3,oasis_prelos])
        elif val < 0.17:
            oasis_prelos = np.nanmax([5,oasis_prelos])
        else:
            oasis_prelos = np.nanmax([np.nan,oasis_prelos])  
    if pd_dataframe['PRELOS'].isnull().all():
        oasis_prelos = np.nan 
    # Age, years
    for val in pd_dataframe['AGE']:
        if val < 24:
            oasis_age = np.nanmax([0,oasis_age])
        elif val >= 24 and val <= 53:
            oasis_age = np.nanmax([3,oasis_age])
        elif val > 53 and val <= 77:
            oasis_age = np.nanmax([6,oasis_age])
        elif val > 77 and val <= 89:
            oasis_age = np.nanmax([9,oasis_age])
        elif val > 89:
            oasis_age = np.nanmax([7,oasis_age])
        else:
            oasis_age = np.nanmax([np.nan,oasis_age])

    if pd_dataframe['AGE'].isnull().all():
        oasis_age = np.nan 

    # Glasgow Coma Scale
    for val in pd_dataframe['GCS_TOTAL']:
        if val == 15:
            oasis_gcs = np.nanmax([0,oasis_gcs])
        elif val == 14:
            oasis_gcs = np.nanmax([3,oasis_gcs])
        elif val >= 8 and val <= 13:
            oasis_gcs = np.nanmax([4,oasis_gcs])
        elif val >= 3 and val <= 7:
            oasis_gcs = np.nanmax([10,oasis_gcs])
        else:
            oasis_gcs = np.nanmax([np.nan,oasis_gcs])
    if pd_dataframe['GCS_TOTAL'].isnull().all():
        oasis_gcs = np.nan
     
    # Heart rate
    for val in pd_dataframe['HRATE']:
        if val >= 33 and val <= 88:
            oasis_hr = np.nanmax([0,oasis_hr])
        elif val > 88 and val <= 106:
            oasis_hr = np.nanmax([1,oasis_hr])
        elif val > 106 and val <= 125:
            oasis_hr = np.nanmax([3,oasis_hr])
        elif val < 33:
            oasis_hr = np.nanmax([4,oasis_hr])
        elif val > 125:
            oasis_hr = np.nanmax([6,oasis_hr])
        else:
            oasis_hr = np.nanmax([np.nan,oasis_hr])
    if pd_dataframe['HRATE'].isnull().all():
        oasis_hr = np.nan
 
    # Mean arterial pressure
    for val in pd_dataframe['MAP']:
        if val >=61.33 and val <= 143.44:
            oasis_map = np.nanmax([0,oasis_map])
        elif val >= 51.0 and val < 61.33:
            oasis_map = np.nanmax([2,oasis_map])
        elif (val >= 20.65 and val < 51.0) or (val > 143.44):
            oasis_map = np.nanmax([3,oasis_map])
        elif val < 20.65:
            oasis_map = np.nanmax([4,oasis_map])
        else:
            oasis_map = np.nanmax([np.nan,oasis_map])
    if pd_dataframe['MAP'].isnull().all():
        oasis_map = np.nan
 
    # Respiratory Rate
    for val in pd_dataframe['RESP_RATE']:
        if val >=13 and val <= 22:
            oasis_resp = np.nanmax([0,oasis_resp])
        elif (val >= 6 and val <= 12) or (val >= 23 and val <= 30):
            oasis_resp = np.nanmax([1,oasis_resp])
        elif val > 30 and val <= 44:
            oasis_resp = np.nanmax([6,oasis_resp])
        elif val > 44:
            oasis_resp = np.nanmax([9,oasis_resp])
        elif val < 6:
            oasis_resp = np.nanmax([10,oasis_resp])
        else:
            oasis_resp = np.nanmax([np.nan,oasis_resp])
    if pd_dataframe['RESP_RATE'].isnull().all():
        oasis_resp = np.nan
 
    # Temperature, C
    for val in pd_dataframe['TEMP_C']:
        if val >= 36.40 and val <= 36.88:
            oasis_temp = np.nanmax([0,oasis_temp])
        elif (val >= 35.94 and val < 36.40) or (val > 36.88 and val <= 39.88):
            oasis_temp = np.nanmax([2,oasis_temp])
        elif val < 33.22:
            oasis_temp = np.nanmax([3,oasis_temp])            
        elif val >= 33.22 and val < 35.94:
            oasis_temp = np.nanmax([4,oasis_temp])
        elif val > 39.88:
            oasis_temp = np.nanmax([6,oasis_temp])
        else:
            oasis_temp = np.nanmax([np.nan,oasis_temp])
    if pd_dataframe['TEMP_C'].isnull().all():
        oasis_temp = np.nan 

    # Urine output, cc/day (total over 24h)
    val = np.max(pd_dataframe['URINE'])
    if val >=2544.0 and val <= 6896.0:
        oasis_urine = np.nanmax([0,oasis_urine])
    elif val >= 1427.0 and val < 2544.0:
        oasis_urine = np.nanmax([1,oasis_urine])
    elif val >= 671.0 and val < 1427.0:
        oasis_urine = np.nanmax([5,oasis_urine])
    elif val > 6896.0:
        oasis_urine = np.nanmax([8,oasis_urine])
    elif val < 671:
        oasis_urine = np.nanmax([10,oasis_urine])
    else:
        oasis_urine = np.nanmax([np.nan,oasis_urine])
    if pd_dataframe['URINE'].isnull().all():
        oasis_urine = np.nan

    # Ventilated y/n
    for val in pd_dataframe['VENTILATED']:
        if val == 'n':
            oasis_vent = np.nanmax([0,oasis_vent])
        elif val == 'y':
            oasis_vent = np.nanmax([9,oasis_vent])
        else:
            oasis_vent = np.nanmax([np.nan,oasis_vent])
    if pd_dataframe['VENTILATED'].isnull().all():
        oasis_vent = np.nan 

    # Elective surgery y/n
    for val in pd_dataframe['ADMISSION_TYPE']:
        if val == 'ELECTIVE':
            oasis_surg = np.nanmax([0,oasis_surg])
        elif val in ['URGENT','EMERGENCY']:
            oasis_surg = np.nanmax([6,oasis_surg])
        else:
            oasis_surg = np.nanmax([np.nan,oasis_surg])
    if pd_dataframe['ADMISSION_TYPE'].isnull().all():
        oasis_surg = np.nan 
    # Return sum
    oasis_score = sum([oasis_prelos, oasis_age, oasis_gcs, oasis_hr, oasis_map, oasis_resp, \
        oasis_temp, oasis_urine, oasis_vent, oasis_surg])
    return oasis_score

In [52]:
# The MIT License

# Copyright (c) 2015 Tom Pollard

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

def compute_oasis_nonNAN(pd_dataframe):
    """
    Takes Pandas DataFrame as an argument and computes Oxford Acute 
    Severity of Illness Score (OASIS) (http://oasisicu.com/)
    
    The DataFrame should include only measurements taken over the first 24h 
    from admission. pd_dataframe should contain the following columns:
    
    'prelos' => Pre-ICU length of stay, hours
    'age' => Age of patient, years
    'GCS_total' => Total Glasgow Coma Scale for patient
    'hrate' => All heart rate measurements
    'MAP' => All mean arterial blood pressure measurements
    'resp_rate' => All respiratory rate measurements
    'temp_c' => All temperature measurements, C
    'urine' => Total urine output over 24 h (note, not consecutive measurements)
    'ventilated' => Is patient ventilated? (y,n)
    'admission_type' => Type of admission (elective, urgent, emergency) 
    
    Reference:
    Johnson AE, Kramer AA, Clifford GD. A new severity of illness scale 
    using a subset of Acute Physiology And Chronic Health Evaluation 
    data elements shows comparable predictive accuracy.
    Crit Care Med. 2013 Jul;41(7):1711-8. doi: 10.1097/CCM.0b013e31828a24fe
    http://www.ncbi.nlm.nih.gov/pubmed/23660729
    """
    
    # 10 variables
    oasis_score, oasis_prelos, oasis_age, oasis_gcs, oasis_hr, oasis_map, oasis_resp, \
        oasis_temp, oasis_urine, oasis_vent, oasis_surg = 0,0,0,0,0,0,0,0,0,0,0   
    # Pre-ICU length of stay, hours
    for val in pd_dataframe['PRELOS']:
        if val >= 4.95 and val <= 24.0:
            oasis_prelos = np.nanmax([0,oasis_prelos])
        elif val > 311.8:
            oasis_prelos = np.nanmax([1,oasis_prelos])
        elif val > 24.0 and val <= 311.8:
            oasis_prelos = np.nanmax([2,oasis_prelos])
        elif val >= 0.17 and val < 4.95:
            oasis_prelos = np.nanmax([3,oasis_prelos])
        elif val < 0.17:
            oasis_prelos = np.nanmax([5,oasis_prelos])
        else:
            oasis_prelos = np.nanmax([np.nan,oasis_prelos])  
    if pd_dataframe['PRELOS'].isnull().all():
        oasis_prelos = np.nan 
    # Age, years
    for val in pd_dataframe['AGE']:
        if val < 24:
            oasis_age = np.nanmax([0,oasis_age])
        elif val >= 24 and val <= 53:
            oasis_age = np.nanmax([3,oasis_age])
        elif val > 53 and val <= 77:
            oasis_age = np.nanmax([6,oasis_age])
        elif val > 77 and val <= 89:
            oasis_age = np.nanmax([9,oasis_age])
        elif val > 89:
            oasis_age = np.nanmax([7,oasis_age])
        else:
            oasis_age = np.nanmax([np.nan,oasis_age])

    if pd_dataframe['AGE'].isnull().all():
        oasis_age = np.nan 

    # Glasgow Coma Scale
    for val in pd_dataframe['GCS_TOTAL']:
        if val == 15:
            oasis_gcs = np.nanmax([0,oasis_gcs])
        elif val == 14:
            oasis_gcs = np.nanmax([3,oasis_gcs])
        elif val >= 8 and val <= 13:
            oasis_gcs = np.nanmax([4,oasis_gcs])
        elif val >= 3 and val <= 7:
            oasis_gcs = np.nanmax([10,oasis_gcs])
        else:
            oasis_gcs = np.nanmax([np.nan,oasis_gcs])
    if pd_dataframe['GCS_TOTAL'].isnull().all():
        oasis_gcs = np.nan
     
    # Heart rate
    for val in pd_dataframe['HRATE']:
        if val >= 33 and val <= 88:
            oasis_hr = np.nanmax([0,oasis_hr])
        elif val > 88 and val <= 106:
            oasis_hr = np.nanmax([1,oasis_hr])
        elif val > 106 and val <= 125:
            oasis_hr = np.nanmax([3,oasis_hr])
        elif val < 33:
            oasis_hr = np.nanmax([4,oasis_hr])
        elif val > 125:
            oasis_hr = np.nanmax([6,oasis_hr])
        else:
            oasis_hr = np.nanmax([np.nan,oasis_hr])
    if pd_dataframe['HRATE'].isnull().all():
        oasis_hr = np.nan
 
    # Mean arterial pressure
    for val in pd_dataframe['MAP']:
        if val >=61.33 and val <= 143.44:
            oasis_map = np.nanmax([0,oasis_map])
        elif val >= 51.0 and val < 61.33:
            oasis_map = np.nanmax([2,oasis_map])
        elif (val >= 20.65 and val < 51.0) or (val > 143.44):
            oasis_map = np.nanmax([3,oasis_map])
        elif val < 20.65:
            oasis_map = np.nanmax([4,oasis_map])
        else:
            oasis_map = np.nanmax([np.nan,oasis_map])
    if pd_dataframe['MAP'].isnull().all():
        oasis_map = np.nan
 
    # Respiratory Rate
    for val in pd_dataframe['RESP_RATE']:
        if val >=13 and val <= 22:
            oasis_resp = np.nanmax([0,oasis_resp])
        elif (val >= 6 and val <= 12) or (val >= 23 and val <= 30):
            oasis_resp = np.nanmax([1,oasis_resp])
        elif val > 30 and val <= 44:
            oasis_resp = np.nanmax([6,oasis_resp])
        elif val > 44:
            oasis_resp = np.nanmax([9,oasis_resp])
        elif val < 6:
            oasis_resp = np.nanmax([10,oasis_resp])
        else:
            oasis_resp = np.nanmax([np.nan,oasis_resp])
    if pd_dataframe['RESP_RATE'].isnull().all():
        oasis_resp = np.nan
 
    # Temperature, C
    for val in pd_dataframe['TEMP_C']:
        if val >= 36.40 and val <= 36.88:
            oasis_temp = np.nanmax([0,oasis_temp])
        elif (val >= 35.94 and val < 36.40) or (val > 36.88 and val <= 39.88):
            oasis_temp = np.nanmax([2,oasis_temp])
        elif val < 33.22:
            oasis_temp = np.nanmax([3,oasis_temp])            
        elif val >= 33.22 and val < 35.94:
            oasis_temp = np.nanmax([4,oasis_temp])
        elif val > 39.88:
            oasis_temp = np.nanmax([6,oasis_temp])
        else:
            oasis_temp = np.nanmax([np.nan,oasis_temp])
    if pd_dataframe['TEMP_C'].isnull().all():
        oasis_temp = np.nan 

    # Urine output, cc/day (total over 24h)
    val = np.max(pd_dataframe['URINE'])
    if val >=2544.0 and val <= 6896.0:
        oasis_urine = np.nanmax([0,oasis_urine])
    elif val >= 1427.0 and val < 2544.0:
        oasis_urine = np.nanmax([1,oasis_urine])
    elif val >= 671.0 and val < 1427.0:
        oasis_urine = np.nanmax([5,oasis_urine])
    elif val > 6896.0:
        oasis_urine = np.nanmax([8,oasis_urine])
    elif val < 671:
        oasis_urine = np.nanmax([10,oasis_urine])
    else:
        oasis_urine = np.nanmax([np.nan,oasis_urine])
    if pd_dataframe['URINE'].isnull().all():
        oasis_urine = np.nan

    # Ventilated y/n
    for val in pd_dataframe['VENTILATED']:
        if val == 'n':
            oasis_vent = np.nanmax([0,oasis_vent])
        elif val == 'y':
            oasis_vent = np.nanmax([9,oasis_vent])
        else:
            oasis_vent = np.nanmax([np.nan,oasis_vent])
    if pd_dataframe['VENTILATED'].isnull().all():
        oasis_vent = np.nan 

    # Elective surgery y/n
    for val in pd_dataframe['ADMISSION_TYPE']:
        if val == 'ELECTIVE':
            oasis_surg = np.nanmax([0,oasis_surg])
        elif val in ['URGENT','EMERGENCY']:
            oasis_surg = np.nanmax([6,oasis_surg])
        else:
            oasis_surg = np.nanmax([np.nan,oasis_surg])
    if pd_dataframe['ADMISSION_TYPE'].isnull().all():
        oasis_surg = np.nan 
    # Return sum
    oasis_score = np.nansum([oasis_prelos, oasis_age, oasis_gcs, oasis_hr, oasis_map, oasis_resp, \
        oasis_temp, oasis_urine, oasis_vent, oasis_surg])
    return oasis_score

In [53]:
def get_oasis_for_single_icustay_id(admission_row, hours): #take argument a Series
    admission = admission_row.to_frame().T #T = transpose series, transform a series to dataframe
    #limpiamos --> quedar con los mesuras tomadas dentro de un dia ---> necesito relacionar con intime --> primero merge to admission
    #temperatura: 
    df_temp_merged = pd.merge(admission,df_temp,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID']) 
    df_temp_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_temp_merged['TEMP_C_TIME'])-pd.to_datetime(df_temp_merged['INTIME'])
    df_temp_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_temp_merged['DAY_1'] = df_temp_merged['DAY_1'].fillna(hours+1) #o fillna(np.nan)??
    #df_temp_merged=df_temp_merged[df_temp_merged['day_1'] <= 24.0]
    df_temp_merged=df_temp_merged[df_temp_merged['DAY_1'] <= hours]

    df_temp_merged=df_temp_merged.drop(['DAY_1','TEMP_C_TIME'], axis=1)
    df_temp_merged['TEMP_C'] = pd.to_numeric(df_temp_merged['TEMP_C'])

    #heart rate:
    df_hrate_merged=pd.merge(admission,df_hrate,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_hrate_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_hrate_merged['HRATE_TIME'])-pd.to_datetime(df_hrate_merged['INTIME'])
    df_hrate_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_hrate_merged['DAY_1'] = df_hrate_merged['DAY_1'].fillna(hours+1)
    #df_hrate_merged=df_hrate_merged[df_hrate_merged['day_1'] <= 24.0]
    df_hrate_merged=df_hrate_merged[df_hrate_merged['DAY_1'] <= hours]
    df_hrate_merged=df_hrate_merged.drop(['DAY_1','HRATE_TIME'], axis=1)

    #respiratory rate
    df_resp_rate_merged=pd.merge(admission,df_resp_rate,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_resp_rate_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_resp_rate_merged['RESP_RATE_TIME'])-pd.to_datetime(df_resp_rate_merged['INTIME'])
    df_resp_rate_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_resp_rate_merged['DAY_1'] = df_resp_rate_merged['DAY_1'].fillna(hours+1)
    #df_resp_rate_merged=df_resp_rate_merged[df_resp_rate_merged['day_1'] <= 24.0]
    df_resp_rate_merged=df_resp_rate_merged[df_resp_rate_merged['DAY_1'] <= hours]
    df_resp_rate_merged=df_resp_rate_merged.drop(['DAY_1','RESP_RATE_TIME'], axis=1)

    #Arterial Blood Pressure mean (MAP)
    df_MAP_merged=pd.merge(admission,df_MAP,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_MAP_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_MAP_merged['MAP_TIME'])-pd.to_datetime(df_MAP_merged['INTIME'])
    df_MAP_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_MAP_merged['DAY_1'] = df_MAP_merged['DAY_1'].fillna(hours+1)
    #df_MAP_merged=df_MAP_merged[df_MAP_merged['day_1'] <= 24.0]
    df_MAP_merged=df_MAP_merged[df_MAP_merged['DAY_1'] <= hours]
    df_MAP_merged=df_MAP_merged.drop(['DAY_1','MAP_TIME'], axis=1)

    #ventilation (invasive and non invasive)
    df_ventilated_merged=pd.merge(admission,df_ventilated,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_ventilated_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_ventilated_merged['STARTTIME'])-pd.to_datetime(df_ventilated_merged['INTIME'])
    df_ventilated_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    #df_ventilated_merged['DAY_1'] = df_ventilated_merged['DAY_1'].fillna(np.nan)
    #df_ventilated_merged=df_ventilated_merged[df_ventilated_merged['day_1'] <= 24.0]
    df_ventilated_merged=df_ventilated_merged[~(df_ventilated_merged['DAY_1'] > hours)]
    df_ventilated_merged=df_ventilated_merged.drop(['DAY_1'], axis=1)

    df_ventilated_merged['VENTILATED'] = df_ventilated_merged['STARTTIME'].apply(lambda x: 'y' if not pd.isnull(x) else 'n')
    df_ventilated_merged=df_ventilated_merged.drop(['STARTTIME'], axis=1)

    

    #GCS_total = GCS_M + GCS_V + GCS_E for metavision
    #GCS_total = GCS  carevue
    df_GCS_E_merged=pd.merge(admission,df_GCS_E,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_GCS_E_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_GCS_E_merged['GCS_E_TIME'])-pd.to_datetime(df_GCS_E_merged['INTIME'])
    df_GCS_E_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_GCS_E_merged['DAY_1'] = df_GCS_E_merged['DAY_1'].fillna(hours+1)
    #df_220739_merged=df_220739_merged[df_220739_merged['day_1'] <= 24.0]
    df_GCS_E_merged=df_GCS_E_merged[df_GCS_E_merged['DAY_1'] <= hours]
    df_GCS_E_merged=df_GCS_E_merged.drop(['DAY_1','GCS_E_TIME'], axis=1)


    df_GCS_M_merged=pd.merge(admission,df_GCS_M,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_GCS_M_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_GCS_M_merged['GCS_M_TIME'])-pd.to_datetime(df_GCS_M_merged['INTIME'])
    df_GCS_M_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_GCS_M_merged['DAY_1'] = df_GCS_M_merged['DAY_1'].fillna(hours+1)
    #df_223901_merged=df_223901_merged[df_223901_merged['day_1'] <= 24.0]
    df_GCS_M_merged=df_GCS_M_merged[df_GCS_M_merged['DAY_1'] <= hours]
    df_GCS_M_merged=df_GCS_M_merged.drop(['DAY_1','GCS_M_TIME'], axis=1)

    df_GCS_V_merged=pd.merge(admission,df_GCS_V,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_GCS_V_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_GCS_V_merged['GCS_V_TIME'])-pd.to_datetime(df_GCS_V_merged['INTIME'])
    df_GCS_V_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_GCS_V_merged['DAY_1'] = df_GCS_V_merged['DAY_1'].fillna(hours+1)
    #df_223900_merged=df_223900_merged[df_223900_merged['day_1'] <= 24.0]
    df_GCS_V_merged=df_GCS_V_merged[df_GCS_V_merged['DAY_1'] <= hours]
    df_GCS_V_merged=df_GCS_V_merged.drop(['DAY_1','GCS_V_TIME'], axis=1)

    df_GCS_TOTAL_CAREVUE_merged=pd.merge(admission,df_GCS_TOTAL_CAREVUE,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_GCS_TOTAL_CAREVUE_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_GCS_TOTAL_CAREVUE_merged['GCS_TOTAL_CAREVUE_TIME'])-pd.to_datetime(df_GCS_TOTAL_CAREVUE_merged['INTIME'])
    df_GCS_TOTAL_CAREVUE_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_GCS_TOTAL_CAREVUE_merged['DAY_1'] = df_GCS_TOTAL_CAREVUE_merged['DAY_1'].fillna(hours+1)
    #df_198_merged=df_198_merged[df_198_merged['day_1'] <= 24.0]
    df_GCS_TOTAL_CAREVUE_merged=df_GCS_TOTAL_CAREVUE_merged[df_GCS_TOTAL_CAREVUE_merged['DAY_1'] <= hours]
    df_GCS_TOTAL_CAREVUE_merged=df_GCS_TOTAL_CAREVUE_merged.drop(['DAY_1','GCS_TOTAL_CAREVUE_TIME'], axis=1)

    #TODO: importa que sean mesurados en el mismo tiempo los GCS, o solo coger el peor y ya esta??????
    #min() de los dataframes, si es vacio retorna nan 
    #si solo esta un de los GCS? No consideramos ese GCS total: np.nan + x + y = np.nan
    GCS_total_metavision = df_GCS_E_merged['GCS_E'].min() + df_GCS_M_merged['GCS_M'].min() + df_GCS_V_merged['GCS_V'].min()
    GCS_total_carevue = df_GCS_TOTAL_CAREVUE_merged['GCS_TOTAL_CAREVUE'].min()


    #total urine in 24 hours
    df_urine_merged=pd.merge(admission,df_urine,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
    df_urine_merged.drop_duplicates()
    day_1_tmp= pd.to_datetime(df_urine_merged['URINE_TIME'])-pd.to_datetime(df_urine_merged['INTIME'])
    df_urine_merged['DAY_1']=np.round(np.abs(day_1_tmp.dt.total_seconds()/3600),1) #hours
    df_urine_merged['DAY_1'] = df_urine_merged['DAY_1'].fillna(hours+1)
    #df_urine_merged=df_urine_merged[df_urine_merged['day_1'] <= 24.0]
    df_urine_merged=df_urine_merged[df_urine_merged['DAY_1'] <= hours]
    df_urine_merged=df_urine_merged.drop(['DAY_1','URINE_TIME'], axis=1)
    #sumamos todo para obtener total urine en 24 horas:
    df_urine_merged['URINE'] = df_urine_merged['URINE'].sum(min_count=1)
    #Quoting from pandas latest docs it says the min_count will be 0 for all-NA series. If you say min_count=1 then the result of the sum will be a nan.
    #print(df_urine_merged['urine'])


    #ultimo paso, concatenar columnas de diferentes variables para obtener un unico dataframe para compute oasis
    df_final_merge = pd.concat([df_temp_merged, df_hrate_merged['HRATE'],df_resp_rate_merged['RESP_RATE'],df_MAP_merged['MAP'],
                              df_ventilated_merged['VENTILATED'],df_urine_merged['URINE']], axis=1) 

    df_final_merge['HRATE'] = pd.to_numeric(df_final_merge['HRATE'])
    
    #obs Unable to parse string ">60/min retracts", hay algunas filas con RESP_RATE = >60/min retracts los considero como 60, ya que OASIS para los >44 les asignan un mismo score 
    
    #df_final_merge['RESP_RATE'] = pd.to_numeric(df_final_merge['RESP_RATE'])
    df_final_merge['RESP_RATE']=df_final_merge['RESP_RATE'].apply(lambda rate: pd.to_numeric(rate) if (rate != '>60/min retracts' and rate != '>60/minute') else 60 )
    df_final_merge['MAP'] = pd.to_numeric(df_final_merge['MAP'])


    if np.isnan(GCS_total_metavision) and np.isnan(GCS_total_carevue):
        GCS_total_min = np.nan
    else:
        GCS_total_min = np.nanmin([GCS_total_metavision,GCS_total_carevue])

    df_final_merge['GCS_TOTAL'] = GCS_total_min


    #print(df_final_merge.shape)
    #print(df_final_merge.isnull().sum())
    #print(df_final_merge.iloc[0:10,12:19] )
    oasis = compute_oasis(df_final_merge)
    oasis_nonan = compute_oasis_nonNAN(df_final_merge)

    #return pd.Series({'OASIS' : oasis})
    return pd.Series({'OASIS': oasis,'OASIS_NONAN': oasis_nonan})


In [54]:
import sys

In [55]:
#Para ejecutar en 3 partes, por si tarda mucho(en total aprox. 6h) y passa algún problema...
#Define dataset for train:
result_24_horas_part1 = pd.DataFrame()
for stay_dir in tqdm(os.listdir(stays_root_path)[0:20000], desc='Iterating over stays'):
    dn = os.path.join(stays_root_path, stay_dir)
    try:
        icustay_id = int(stay_dir)
        if not os.path.isdir(dn):
            raise Exception
    except:
        continue

    try:
        # reading tables of this icustay_id
        stays = read_stays(os.path.join(stays_root_path, stay_dir))
        df_temp,df_hrate,df_resp_rate,df_MAP,df_GCS_E,df_GCS_M,df_GCS_V,df_GCS_TOTAL_CAREVUE = read_events(os.path.join(stays_root_path, stay_dir))
        df_ventilated = read_ventilation(os.path.join(stays_root_path, stay_dir))
        df_urine = read_urine(os.path.join(stays_root_path, stay_dir))
    
    except:
        sys.stderr.write('Error reading from disk for icustay_id: {}\n'.format(icustay_id))
        continue
    
    stays_oasis = stays.apply(lambda x: get_oasis_for_single_icustay_id(x, 24.0), axis=1) 
    result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))

Iterating over stays:   0%|                              | 0/137 [00:00<?, ?it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:   1%|▏                     | 1/137 [00:00<00:54,  2.49it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:   1%|▎                     | 2/137 [00:00<00:38,  3.53it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.appe

Iterating over stays:  18%|███▊                 | 25/137 [00:05<00:26,  4.28it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  19%|███▉                 | 26/137 [00:05<00:28,  3.88it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  20%|████▏                | 27/137 [00:05<00:26,  4.21it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.appe

Iterating over stays:  36%|███████▌             | 49/137 [00:10<00:16,  5.20it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  36%|███████▋             | 50/137 [00:10<00:17,  5.10it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  37%|███████▊             | 51/137 [00:10<00:16,  5.16it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.appe

Iterating over stays:  53%|███████████▏         | 73/137 [00:14<00:12,  5.11it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  54%|███████████▎         | 74/137 [00:15<00:13,  4.75it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  55%|███████████▍         | 75/137 [00:15<00:12,  4.81it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.appe

Iterating over stays:  71%|██████████████▊      | 97/137 [00:19<00:07,  5.31it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  72%|███████████████      | 98/137 [00:19<00:07,  5.07it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  72%|███████████████▏     | 99/137 [00:19<00:07,  5.26it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.appe

Iterating over stays:  88%|█████████████████▋  | 121/137 [00:24<00:03,  5.15it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  89%|█████████████████▊  | 122/137 [00:24<00:02,  5.14it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.append(pd.concat([stays, stays_oasis], axis=1))
Iterating over stays:  90%|█████████████████▉  | 123/137 [00:24<00:02,  5.22it/s]/tmp/ipykernel_2453/3610529308.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_24_horas_part1 = result_24_horas_part1.appe

In [56]:
result_24_horas_part1.to_csv('result_24_horas_part1.csv')

In [57]:

#Para ejecutar en 3 partes:
#Define dataset for train:
result_24_horas_part2 = pd.DataFrame()

for stay_dir in tqdm(os.listdir(stays_root_path)[20000:40000], desc='Iterating over stays'):
    dn = os.path.join(stays_root_path, stay_dir)
    try:
        icustay_id = int(stay_dir)
        if not os.path.isdir(dn):
            raise Exception
    except:
        continue

    try:
        # reading tables of this icustay_id
        stays = read_stays(os.path.join(stays_root_path, stay_dir))
        df_temp,df_hrate,df_resp_rate,df_MAP,df_GCS_E,df_GCS_M,df_GCS_V,df_GCS_TOTAL_CAREVUE = read_events(os.path.join(stays_root_path, stay_dir))
        df_ventilated = read_ventilation(os.path.join(stays_root_path, stay_dir))
        df_urine = read_urine(os.path.join(stays_root_path, stay_dir))
    except:
        sys.stderr.write('Error reading from disk for icustay_id: {}\n'.format(icustay_id))
        continue

        
    stays_oasis = stays.apply(lambda x: get_oasis_for_single_icustay_id(x, 24.0), axis=1) 
    result_24_horas_part2 = result_24_horas_part2.append(pd.concat([stays, stays_oasis], axis=1))
    


Iterating over stays: 0it [00:00, ?it/s]


In [58]:
result_24_horas_part2.to_csv('result_24_horas_part2.csv')

In [59]:

#Para ejecutar en 3 partes:
#Define dataset for train:
result_24_horas_part3 = pd.DataFrame()

for stay_dir in tqdm(os.listdir(stays_root_path)[40000:], desc='Iterating over stays'):
    dn = os.path.join(stays_root_path, stay_dir)
    try:
        icustay_id = int(stay_dir)
        if not os.path.isdir(dn):
            raise Exception
    except:
        continue

    try:
        # reading tables of this icustay_id
        stays = read_stays(os.path.join(stays_root_path, stay_dir))
        df_temp,df_hrate,df_resp_rate,df_MAP,df_GCS_E,df_GCS_M,df_GCS_V,df_GCS_TOTAL_CAREVUE = read_events(os.path.join(stays_root_path, stay_dir))
        df_ventilated = read_ventilation(os.path.join(stays_root_path, stay_dir))
        df_urine = read_urine(os.path.join(stays_root_path, stay_dir))
    except:
        sys.stderr.write('Error reading from disk for icustay_id: {}\n'.format(icustay_id))
        continue

        
    stays_oasis = stays.apply(lambda x: get_oasis_for_single_icustay_id(x, 24.0), axis=1) 
    result_24_horas_part3 = result_24_horas_part3.append(pd.concat([stays, stays_oasis], axis=1))
    

Iterating over stays: 0it [00:00, ?it/s]


In [60]:
result_24_horas_part3.to_csv('result_24_horas_part3.csv')

In [19]:
result_24_horas_part1=pd.read_csv('result_24_horas_part1.csv')
result_24_horas_part2=pd.read_csv('result_24_horas_part2.csv')
result_24_horas_part3=pd.read_csv('result_24_horas_part3.csv')

In [20]:
result_24_horas_final = pd.concat([result_24_horas_part1, result_24_horas_part2,result_24_horas_part3], ignore_index=True)

In [ ]:
#result_24_horas_final = pd.concat([result_24_horas_part1, result_24_horas_part2], ignore_index=True)

In [21]:
result_24_horas_final.to_csv('result_24_horas_final.csv')

In [2]:
result_24_horas_final=pd.read_csv('result_24_horas_final.csv')

In [3]:
result_24_horas_final.isnull().sum()

Unnamed: 0            0
Unnamed: 0.1          0
SUBJECT_ID            0
HADM_ID               0
ADMITTIME             0
ADMISSION_TYPE        0
ICUSTAY_ID            0
INTIME                0
LOS                  10
AGE                   0
PRELOS                0
OASIS             13655
OASIS_NONAN           0
dtype: int64